In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore

cache_dir = LocalFileStore("./.cache/")



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n\n",
    chunk_size=600,
    chunk_overlap=100
)

loader = UnstructuredFileLoader("./files/Chapter_one.txt", encoding='utf8')

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

In [21]:
results = vectorstore.similarity_search("where does winston live")

len(results)

[Document(page_content="Winston repaired the sink, taking out the unpleasant knot of hair that was stopping the water running away. He washed his hands and went back to the other room.\n\n'Put your hands up!' shouted a voice.\n\nA big, handsome boy of nine was pointing a toy gun at him. His small sister, about two years younger, pointed a piece of wood. Both were dressed in the blue, grey and red uniforms of the Spies. Winston put his hands up. The look of hate on the boy's face made him feel that it was not quite a game.\n\n'You're a Eurasian spy!' screamed the boy. ’You're a thoughtcriminaUl'U shoot you, I'll vaporize you!'\n\nSuddenly they were both running round him, shouting 'Spy! Thoughtcriminal!' The little girl did everything seconds after her older brother did it. It was frightening, like the games of young, dangerous wild animals that will soon be man-eaters. Winston could see that the boy really wanted to hit or kick him, and was\n\n9\n\nnearly big enough to do so. He was gl

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

In [13]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [14]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase.Do NOT reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

In [16]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "",
        ),
        ("human", "I want to specialized in explaining {cuisine} Haikus."),
    ]
)

chef_chain = chef_prompt | chat

In [18]:
veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a specialized in writing Haikus about programming languages.",
        ),
        ("human", "{recipe}"),
    ]
)


veg_chain = veg_chef_prompt | chat


final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "Python"})

That's a unique and interesting specialization! Python haikus can be a fun way to combine programming and poetry. Haikus typically follow a 5-7-5 syllable pattern, so you can create haikus that capture the essence of Python programming concepts in a concise and creative way. Feel free to share any Python haikus you've written or ask for assistance in crafting them!Thank you! Here's a Python haiku for you:

Indentation rules,
Pythonic code flows with ease,
Guido's gift to us.

AIMessageChunk(content="Thank you! Here's a Python haiku for you:\n\nIndentation rules,\nPythonic code flows with ease,\nGuido's gift to us.")

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "do you know about the movie Dune: Part Two?",
        "answer": """
        Here is what I know:
        Director : Denis Villeneuve
        Stars : Timothée Chalamet, Zendaya, Rebecca Ferguson
        Storyline: Paul Atreides unites with Chani and the Fremen while on a warpath of revenge against the conspirators who destroyed his family. Facing a choice between the love of his life and the fate of the known universe, he endeavors to prevent a terrible future only he can foresee
        Genres: Action, Adventure, Drama, Sci-Fi
        """,
    },
    {
        "question": "do you know about the movie Original title: The Dark Knight?",
        "answer": """
        I know this:
        Director : Christopher Nolan
        Stars : Christian Bale, Heath Ledger, Aaron Eckhart
        Storyline: Set within a year after the events of Batman Begins (2005), Batman, Lieutenant James Gordon, and new District Attorney Harvey Dent successfully begin to round up the criminals that plague Gotham City, until a mysterious and sadistic criminal mastermind known only as "The Joker" appears in Gotham, creating a new wave of chaos. Batman's struggle against The Joker becomes deeply personal, forcing him to "confront everything he believes" and improve his technology to stop him. A love triangle develops between Bruce Wayne, Dent, and Rachel Dawes
        Genres : Action, Crime, Drama, Thriller
        """,
    },
    {
        "question": "do you know about the movie Original title: Inception?",
        "answer": """
        I know this:
        Director : Christopher Nolan
        Stars : Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page
        Storyline: Dom Cobb is a skilled thief, the absolute best in the dangerous art of extraction, stealing valuable secrets from deep within the subconscious during the dream state, when the mind is at its most vulnerable. Cobb's rare ability has made him a coveted player in this treacherous new world of corporate espionage, but it has also made him an international fugitive and cost him everything he has ever loved. Now Cobb is being offered a chance at redemption. One last job could give him his life back but only if he can accomplish the impossible, inception. Instead of the perfect heist, Cobb and his team of specialists have to pull off the reverse: their task is not to steal an idea, but to plant one. If they succeed, it could be the perfect crime. But no amount of careful planning or expertise can prepare the team for the dangerous enemy that seems to predict their every move. An enemy that only Cobb could have seen coming
        Genres : Action, Adventure, Sci-Fi, Thriller
        """,
    },
]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: do you know about the movie {movie}?",
    input_variables=["movie"],
)

chain = prompt | chat

chain.invoke({"movie": "Original title: Avengers: Infinity War"})

AI:
        I know this:
        Directors : Anthony Russo, Joe Russo
        Stars : Robert Downey Jr., Chris Hemsworth, Mark Ruffalo
        Storyline: As the Avengers and their allies have continued to protect the world from threats too large for any one hero to handle, a new danger has emerged from the cosmic shadows: Thanos. A despot of intergalactic infamy, his goal is to collect all six Infinity Stones, artifacts of unimaginable power, and use them to inflict his twisted will on all of reality. Everything the Avengers have fought for has led up to this moment - the fate of Earth and existence itself has never been more uncertain
        Genres : Action, Adventure, Sci-Fi

AIMessageChunk(content='AI:\n        I know this:\n        Directors : Anthony Russo, Joe Russo\n        Stars : Robert Downey Jr., Chris Hemsworth, Mark Ruffalo\n        Storyline: As the Avengers and their allies have continued to protect the world from threats too large for any one hero to handle, a new danger has emerged from the cosmic shadows: Thanos. A despot of intergalactic infamy, his goal is to collect all six Infinity Stones, artifacts of unimaginable power, and use them to inflict his twisted will on all of reality. Everything the Avengers have fought for has led up to this moment - the fate of Earth and existence itself has never been more uncertain\n        Genres : Action, Adventure, Sci-Fi')